# Data Wrangling
This notebook imports data and cleans it.

# Imports

In [1]:
import pandas as pd
from ydata_profiling import ProfileReport

# Orders

In [3]:
data_path = './../data/raw/'

orders_df = pd.read_csv(data_path + 'orders.csv')
orders_df.head()

,Order,Order Date,Currency,Order Subtotal,Order Shipping Price,Order Tax Total,Order Total,Order Refunded Amount,Fulfillment Date,Fulfillment Type,...,Recipient Region,Recipient Country,Item Quantity,Item Name,Item SKU,Item Variation,Item Modifiers,Item Price,Item Options Total Price,Item Total Price
0,Uber Eats Delivery 8F819,2024/12/18,USD,37.90,NaN,3.46,41.36,NaN,"12/18/2024, 1:58 PM",Pickup,...,NaN,NaN,1,CRISPY CHICKEN SANDWICH,NaN,Regular,NaN,16.46,16.46,18.02
1,Uber Eats Delivery 8F819,2024/12/18,USD,37.90,NaN,3.46,41.36,NaN,"12/18/2024, 1:58 PM",Pickup,...,NaN,NaN,1,LOADED FRIES,NaN,Regular,"1 x Signature BBQ Sauce Drizzle, 1 x Salt and ...",11.99,12.74,13.95
2,Uber Eats Delivery 8F819,2024/12/18,USD,37.90,NaN,3.46,41.36,NaN,"12/18/2024, 1:58 PM",Pickup,...,NaN,NaN,1,CREAMY BLUE CHEESE DIP,NaN,Regular,NaN,1.45,1.45,1.45
3,Uber Eats Delivery 8F819,2024/12/18,USD,37.90,NaN,3.46,41.36,NaN,"12/18/2024, 1:58 PM",Pickup,...,NaN,NaN,1,ARTISAN MAC AND CHEESE,NaN,Side,NaN,7.25,7.25,7.94
4,Uber Eats Delivery 7751A,2024/12/17,USD,113.09,NaN,10.74,123.83,NaN,"12/17/2024, 7:56 PM",Pickup,...,NaN,NaN,3,BEEF SHORT RIB,NaN,Regular,NaN,33.95,33.95,111.52


In [4]:
orders_df.columns

Index(['Order', 'Order Date', 'Currency', 'Order Subtotal',
       'Order Shipping Price', 'Order Tax Total', 'Order Total',
       'Order Refunded Amount', 'Fulfillment Date', 'Fulfillment Type',
       'Fulfillment Status', 'Channels', 'Fulfillment Location',
       'Fulfillment Notes', 'Recipient Name', 'Recipient Email',
       'Recipient Phone', 'Recipient Address', 'Recipient Address 2',
       'Recipient Postal Code', 'Recipient City', 'Recipient Region',
       'Recipient Country', 'Item Quantity', 'Item Name', 'Item SKU',
       'Item Variation', 'Item Modifiers', 'Item Price',
       'Item Options Total Price', 'Item Total Price'],
      dtype='object')

In [ ]:
# Generate initial report of orders.csv

profile = ProfileReport(orders_df, title='Orders Profiling Report')
profile.to_file('./../reports/profile-reports/orders-profile-report.html')

# Items

In [3]:
items_df = pd.read_csv(data_path + 'items.csv')
items_df.head()

,Date,Time,Time Zone,Category,Item,Qty,Price Point Name,SKU,Modifiers Applied,Gross Sales,...,Location,Dining Option,Customer ID,Customer Name,Customer Reference ID,Unit,Count,Itemization Type,Fulfillment Note,Token
0,2023-12-31,17:48:40,Pacific Time (US & Canada),Mains,BELLY SLIDERS (Voided),1.0,6 sliders,NaN,NaN,$0.00,...,Belly Rubb,For Here,NaN,NaN,NaN,ea,1,Physical Good,NaN,9614A912-0D13-4E3A-85F5-E7115915F515
1,2023-12-31,17:48:40,Pacific Time (US & Canada),Mains,BELLY SLIDERS (Voided),1.0,4 sliders,NaN,NaN,$0.00,...,Belly Rubb,For Here,NaN,NaN,NaN,ea,1,Physical Good,NaN,066F4B7E-69FB-4554-88DA-99ECE8BEEB6A
2,2023-12-31,17:48:40,Pacific Time (US & Canada),Mains,GET YOUR BABY BACK! (Voided),2.0,Full Rack,NaN,NaN,$0.00,...,Belly Rubb,For Here,NaN,NaN,NaN,ea,2,Physical Good,NaN,4D4021CE-9753-4704-9A59-EC94859819B3
3,2023-12-31,17:48:40,Pacific Time (US & Canada),Sides,MAC&CHEESE (Voided),1.0,Full,NaN,NaN,$0.00,...,Belly Rubb,For Here,NaN,NaN,NaN,ea,1,Physical Good,NaN,UFAF57JBKMKJN6SZNS2MCH4M
4,2023-12-31,17:42:54,Pacific Time (US & Canada),Mains,BELLY SLIDERS,1.0,6 sliders,NaN,NaN,$22.15,...,Belly Rubb,For Here,Q170DHBMPCY9K1ZZ49P049BCER,Narek Ekmekjyan,NaN,ea,1,Physical Good,NaN,9614A912-0D13-4E3A-85F5-E7115915F515


In [14]:
orders_df = pd.read_csv(data_path + 'orders.csv')
orders_df.head()

,Order,Order Date,Currency,Order Subtotal,Order Shipping Price,Order Tax Total,Order Total,Order Refunded Amount,Fulfillment Date,Fulfillment Type,...,Recipient Region,Recipient Country,Item Quantity,Item Name,Item SKU,Item Variation,Item Modifiers,Item Price,Item Options Total Price,Item Total Price
0,Uber Eats Delivery 8F819,2024/12/18,USD,37.90,NaN,3.46,41.36,NaN,"12/18/2024, 1:58 PM",Pickup,...,NaN,NaN,1,CRISPY CHICKEN SANDWICH,NaN,Regular,NaN,16.46,16.46,18.02
1,Uber Eats Delivery 8F819,2024/12/18,USD,37.90,NaN,3.46,41.36,NaN,"12/18/2024, 1:58 PM",Pickup,...,NaN,NaN,1,LOADED FRIES,NaN,Regular,"1 x Signature BBQ Sauce Drizzle, 1 x Salt and ...",11.99,12.74,13.95
2,Uber Eats Delivery 8F819,2024/12/18,USD,37.90,NaN,3.46,41.36,NaN,"12/18/2024, 1:58 PM",Pickup,...,NaN,NaN,1,CREAMY BLUE CHEESE DIP,NaN,Regular,NaN,1.45,1.45,1.45
3,Uber Eats Delivery 8F819,2024/12/18,USD,37.90,NaN,3.46,41.36,NaN,"12/18/2024, 1:58 PM",Pickup,...,NaN,NaN,1,ARTISAN MAC AND CHEESE,NaN,Side,NaN,7.25,7.25,7.94
4,Uber Eats Delivery 7751A,2024/12/17,USD,113.09,NaN,10.74,123.83,NaN,"12/17/2024, 7:56 PM",Pickup,...,NaN,NaN,3,BEEF SHORT RIB,NaN,Regular,NaN,33.95,33.95,111.52


In [11]:
items_df['Customer Name'].value_counts()

Customer Name
Narek Ekmekjyan       8
Arus Keshishyan       4
Alissa Khachmanyan    4
Michael Keshishyan    3
Emily Samvalian       3
Name: count, dtype: int64

In [10]:
item_sales_df = pd.read_csv(data_path + 'item-sales-summary.csv')
item_sales_df.head()

,Item Name,Item Variation,SKU,Category,Items Sold,Gross Sales,Items Refunded,Refunds,Discounts & Comps,Net Sales,Tax,Unit,Units Sold,Units Refunded
0,BELLY SLIDERS,2 sliders,NaN,Uncategorized,1,$9.98,0,$0.00,$0.00,$9.98,$0.95,ea,1,0
1,BBQ dipping sauce,Regular,NaN,Condiments,3,$2.55,0,$0.00,$0.00,$2.55,$0.00,ea,3,0
2,SIGNATURE BBQ SAUCE,Regular,NaN,Condiments,2,$1.70,0,$0.00,$0.00,$1.70,$0.16,ea,2,0
3,BELLY SLIDERS,4 sliders,NaN,Mains,2,$30.84,0,$0.00,$0.00,$30.84,$2.69,ea,2,0
4,BELLY SLIDERS,6 sliders,NaN,Mains,1,$22.15,0,$0.00,$0.00,$22.15,$2.10,ea,1,0


In [4]:
customers_df = pd.read_csv(data_path + 'customer-directory.csv')
customers_df.head()

,Reference ID,First Name,Last Name,Email Address,Phone Number,Nickname,Company Name,Street Address 1,Street Address 2,City,...,Birthday,Memo,Square Customer ID,Creation Source,First Visit,Last Visit,Transaction Count,Total Spend,Email Subscription Status,Instant Profile
0,NaN,Ahlennah,Belton,eowyn797@gmail.com,'+18056996797,NaN,NaN,260 West Elmwood Avenue,Apt K,Burbank,...,NaN,NaN,RN17897VKN317B63F7WDRD0YFC,Merge,2024-04-24,2024-04-24,1,$56.69,unsubscribed,No
1,NaN,Alan L,Kaplan,manowa72@gmail.com,'+18186323824,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,J4ZV8PCYETVR99SG3380RJ5HKG,Instant Profile (via Payment),2024-11-16,2024-11-16,1,$28.97,unknown,Yes
2,NaN,Alejandra,Mejia,aledrm84@gmail.com,'+13235121733,NaN,NaN,5633 Colfax Avenue,Leasing office,Los Angeles,...,NaN,NaN,BRET0KG4Q127HFWN9MP5HQR820,Merge,2024-05-31,2024-05-31,1,$30.98,unknown,No
3,NaN,Alex,NaN,NaN,'+16266164211,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,MEKXVSXHDBBCKZBQ9CX8MB26ZW,Directory,2024-03-30,2024-03-30,1,NaN,NaN,No
4,NaN,Alexander,Diaz,alex.anthony.diaz@gmail.com,'+16266164211,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,JCJ1X244D95FZ8JYTX0VZEKCH8,Merge,2024-03-15,2024-09-17,14,$448.89,unknown,No
